In [115]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import geopandas as gpd

In [116]:
nbhd_transit = pd.read_csv('Overall_nbhd_transit.csv', index_col=0)
nbhd_transit

,neighborhood,geometry,1206,1209,1301,1306,1309,1401,1406,1409,...,2301,2306,2309,2310,2401,2406,2409,2501,no_change,no_transit
0,NORTH CITY,MULTIPOLYGON (((6258808.013999999 1938466.9907...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
1,SAN DIEGO,MULTIPOLYGON (((6293794.477249995 1801763.5917...,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,1,0
2,NESTOR,MULTIPOLYGON (((6303846.415250003 1792939.9542...,98,98,98,98,98,98,98,100,...,99,99,99,99,99,99,99,99,0,0
3,BIRDLAND,"POLYGON ((6284667.651999995 1874418.894749999,...",53,53,53,53,53,53,53,53,...,50,50,50,50,50,50,50,50,0,0
4,CHEROKEE POINT,POLYGON ((6297431.051499993 1853639.0847499967...,32,32,32,32,32,32,32,32,...,51,51,51,51,51,51,51,51,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,TORREY HIGHLANDS,"MULTIPOLYGON (((6287136.998999998 1934267.625,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
126,RANCHO PENASQUITOS,MULTIPOLYGON (((6306168.141249999 1942019.0699...,46,46,46,45,45,45,42,42,...,40,40,40,40,40,40,40,40,0,0
127,SAN PASQUAL,MULTIPOLYGON (((6341872.9537332235 1982808.000...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
128,TIJUANA RIVER VALLEY,"MULTIPOLYGON (((6307704 1784860.0002499968, 63...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1


In [117]:

zhvi_df = pd.read_csv("sd_zhvi_2000_to_2025_nhbd.csv")
zhvi_df.drop(['RegionType', 'StateName', 'State', 'City', 'Metro', 'CountyName'], axis=1, inplace=True)
zhvi_cols = zhvi_df.columns[3:]
obs_started = zhvi_df[zhvi_cols].notnull().cumsum(axis=1) > 0
incomp_obs = zhvi_df[zhvi_cols].isnull() & obs_started
nbhds_incomp = incomp_obs.any(axis=1)
problem_rows = zhvi_df.loc[nbhds_incomp]
rows_clean = ~incomp_obs.any(axis=1)
clean_rows_df = zhvi_df.loc[rows_clean]
def quarter_nonnull(row):
    first_valid = row.first_valid_index()
    if first_valid is None:
        return False

    start = row.index.get_loc(first_valid) + 1
    tb_complete = row.iloc[start:]

    for i in range(0, len(tb_complete), 3):
        group = tb_complete.iloc[i:i + 3]
        if group.notnull().sum() == 0:
            return False 
    return True

mask_valids = zhvi_df[zhvi_cols].apply(quarter_nonnull, axis=1)
clean_zhvi = zhvi_df.loc[mask_valids]
clean_zhvi
clean_zhvi['RegionName'] = clean_zhvi['RegionName'].str.upper()

We need to standardize neighborhood names

In [118]:
clean_zhvi['RegionName'] = clean_zhvi['RegionName'].replace({'JOMACHA-LOMITA': 'JAMACHA LOMITA', 'MT. HOPE': 'MT HOPE', 'BANKERS HILL': 'PARK WEST', 'BIRD LAND': 'BIRDLAND', 'RIDGEVIEW-WEBSTER': 'RIDGEVIEW/WEBSTER', 'CORTEZ HILL': 'CORTEZ', 'RANCHO ENCANTADO': 'RANCHO ENCANTADA', 'GASLAMP QUARTER': 'GASLAMP', 'AZALEA-HOLLYWOOD PARK': 'AZALEA/HOLLYWOOD PARK', 'ROSEVILLE - FLEET RIDGE': 'ROSEVILLE / FLEET RIDGE'})

NEIGHBORHOOD - BROADWAY HEIGHTS NOT COUNTED? Lack of Transit & Lack of Home Ownership and Pricing Data - African-American community in San Diego

In [119]:
for col in clean_zhvi.columns:
    if str(col)[0].isdigit() == True:
        year = int(str(col)[:4])
        month = int(str(col)[5:7])
        if year < 2012:
            clean_zhvi = clean_zhvi.drop(col, axis=1)
        elif year == 2012 and month < 6:
            clean_zhvi = clean_zhvi.drop(col, axis=1)
    

In [120]:
diff_1 = set(list(nbhd_transit['neighborhood'])) -  set(list(clean_zhvi['RegionName']))
diff_1 = list(diff_1)
nbhd_transit = nbhd_transit[~nbhd_transit['neighborhood'].isin(diff_1)]

In [121]:
yymm_cols = [col for col in nbhd_transit.columns if str(col)[0].isdigit()]
transit_long = pd.melt(nbhd_transit, id_vars=['neighborhood', 'geometry', 'no_change', 'no_transit'], value_vars=yymm_cols)
transit_long = transit_long.rename(columns={'variable': 'yymm', 'value': 'trans_score'})

In [122]:
def rename_col(col):
    col = str(col)
    if col[0].isdigit() == True:
        return (col[2:4] + col[5:7])
    else:
        return col

clean_zhvi = clean_zhvi.rename(columns=lambda col: rename_col(col))
clean_zhvi

,RegionID,SizeRank,RegionName,1206,1207,1208,1209,1210,1211,1212,...,2406,2407,2408,2409,2410,2411,2412,2501,2502,2503
0,116625,90,MIRA MESA,3.320540e+05,3.342039e+05,3.372567e+05,3.411879e+05,3.457765e+05,3.505876e+05,3.556491e+05,...,1.046746e+06,1.051713e+06,1.054413e+06,1.057081e+06,1.057116e+06,1.056057e+06,1.054485e+06,1.052457e+06,1.050504e+06,1.046569e+06
1,117557,187,RANCHO PENASQUITOS,4.917199e+05,4.939935e+05,4.972454e+05,5.010422e+05,5.053695e+05,5.101189e+05,5.157166e+05,...,1.450381e+06,1.458144e+06,1.459420e+06,1.460119e+06,1.457302e+06,1.454203e+06,1.450405e+06,1.448254e+06,1.448124e+06,1.445833e+06
2,273140,204,CARMEL VALLEY,6.741591e+05,6.765224e+05,6.811243e+05,6.856790e+05,6.919884e+05,6.980873e+05,7.049418e+05,...,1.943495e+06,1.947692e+06,1.947647e+06,1.949457e+06,1.953105e+06,1.959504e+06,1.968405e+06,1.977301e+06,1.987321e+06,1.994149e+06
3,118673,207,UNIVERSITY CITY,3.971096e+05,3.991437e+05,4.020890e+05,4.050839e+05,4.094694e+05,4.140863e+05,4.194913e+05,...,1.042178e+06,1.044426e+06,1.043921e+06,1.043090e+06,1.040771e+06,1.038827e+06,1.037929e+06,1.037082e+06,1.038156e+06,1.037566e+06
4,275127,253,RANCHO BERNARDO,3.638746e+05,3.662411e+05,3.693218e+05,3.723009e+05,3.766200e+05,3.809945e+05,3.860768e+05,...,1.020553e+06,1.024781e+06,1.025505e+06,1.025162e+06,1.023944e+06,1.024404e+06,1.026353e+06,1.028928e+06,1.030552e+06,1.028547e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,403241,13905,ISLENAIR,2.182529e+05,2.205040e+05,2.224976e+05,2.238268e+05,2.275268e+05,2.328390e+05,2.397080e+05,...,7.341372e+05,7.366393e+05,7.371894e+05,7.369874e+05,7.350599e+05,7.345114e+05,7.391456e+05,7.435692e+05,7.479063e+05,7.449176e+05
109,268215,14394,HORTON PLAZA,1.035187e+06,1.038199e+06,1.045805e+06,1.055482e+06,1.070034e+06,1.087102e+06,1.102909e+06,...,1.777298e+06,1.769644e+06,1.760986e+06,1.755845e+06,1.753376e+06,1.752594e+06,1.744367e+06,1.735349e+06,1.727009e+06,1.720785e+06
110,274893,15337,OLD TOWN,5.197125e+05,5.220849e+05,5.239969e+05,5.263528e+05,5.350053e+05,5.464542e+05,5.587891e+05,...,1.124527e+06,1.119405e+06,1.112519e+06,1.109964e+06,1.105037e+06,1.103857e+06,1.101063e+06,1.096667e+06,1.089447e+06,1.082324e+06
111,403244,16412,BURLINGAME,6.306015e+05,6.308079e+05,6.317689e+05,6.334135e+05,6.406030e+05,6.511496e+05,6.669106e+05,...,1.508703e+06,1.511324e+06,1.510621e+06,1.505252e+06,1.497816e+06,1.495730e+06,1.496153e+06,1.489847e+06,1.477947e+06,1.462991e+06


In [123]:
yymm_cols = [col for col in clean_zhvi.columns if str(col)[0].isdigit()]

zhvi_long = pd.melt(clean_zhvi, id_vars=['RegionID', 'SizeRank', 'RegionName'], value_vars=yymm_cols)
zhvi_long = zhvi_long.rename(columns={'RegionName':'neighborhood', 'variable':'yymm', 'value':'ZHVI'})
zhvi_long

,RegionID,SizeRank,neighborhood,yymm,ZHVI
0,116625,90,MIRA MESA,1206,3.320540e+05
1,117557,187,RANCHO PENASQUITOS,1206,4.917199e+05
2,273140,204,CARMEL VALLEY,1206,6.741591e+05
3,118673,207,UNIVERSITY CITY,1206,3.971096e+05
4,275127,253,RANCHO BERNARDO,1206,3.638746e+05
...,...,...,...,...,...
17397,403241,13905,ISLENAIR,2503,7.449176e+05
17398,268215,14394,HORTON PLAZA,2503,1.720785e+06
17399,274893,15337,OLD TOWN,2503,1.082324e+06
17400,403244,16412,BURLINGAME,2503,1.462991e+06


In [129]:
nbhd_transit

,neighborhood,geometry,1206,1209,1301,1306,1309,1401,1406,1409,...,2301,2306,2309,2310,2401,2406,2409,2501,no_change,no_transit
0,NORTH CITY,MULTIPOLYGON (((6258808.013999999 1938466.9907...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
2,NESTOR,MULTIPOLYGON (((6303846.415250003 1792939.9542...,98,98,98,98,98,98,98,100,...,99,99,99,99,99,99,99,99,0,0
3,BIRDLAND,"POLYGON ((6284667.651999995 1874418.894749999,...",53,53,53,53,53,53,53,53,...,50,50,50,50,50,50,50,50,0,0
4,CHEROKEE POINT,POLYGON ((6297431.051499993 1853639.0847499967...,32,32,32,32,32,32,32,32,...,51,51,51,51,51,51,51,51,0,0
5,LOMA PORTAL,POLYGON ((6263446.993000001 1854500.0890000015...,44,44,44,44,44,44,44,44,...,44,44,44,44,44,44,44,44,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,TIERRASANTA,MULTIPOLYGON (((6329750.657499999 1898434.6107...,22,22,22,22,22,22,22,22,...,22,22,22,22,22,22,22,22,1,0
124,BLACK MOUNTAIN RANCH,MULTIPOLYGON (((6294983.314500004 1951720.0649...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
125,TORREY HIGHLANDS,"MULTIPOLYGON (((6287136.998999998 1934267.625,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
126,RANCHO PENASQUITOS,MULTIPOLYGON (((6306168.141249999 1942019.0699...,46,46,46,45,45,45,42,42,...,40,40,40,40,40,40,40,40,0,0


In [125]:
zhvi_transit = pd.merge(zhvi_long[['neighborhood', 'yymm', 'ZHVI']], transit_long[['neighborhood', 'yymm', 'trans_score']], on=['neighborhood', 'yymm'], how='left')
zhvi_tranit = zhvi_transit.sort_values(['neighborhood', 'yymm'])
zhvi_transit['trans_score'] = zhvi_transit.groupby('neighborhood')['trans_score'].ffill()
zhvi_transit

,neighborhood,yymm,ZHVI,trans_score
0,MIRA MESA,1206,3.320540e+05,116.0
1,RANCHO PENASQUITOS,1206,4.917199e+05,46.0
2,CARMEL VALLEY,1206,6.741591e+05,0.0
3,UNIVERSITY CITY,1206,3.971096e+05,109.0
4,RANCHO BERNARDO,1206,3.638746e+05,51.0
...,...,...,...,...
17440,ISLENAIR,2503,7.449176e+05,6.0
17441,HORTON PLAZA,2503,1.720785e+06,88.0
17442,OLD TOWN,2503,1.082324e+06,95.0
17443,BURLINGAME,2503,1.462991e+06,4.0


In [130]:
const_zhvi = clean_zhvi[['RegionName', 'SizeRank', 'RegionID']].rename(columns={'RegionName':'neighborhood'})
const_trans = nbhd_transit[['neighborhood', 'geometry', 'no_change', 'no_transit']]

zhvi_transit = pd.merge(zhvi_transit, const_zhvi, on='neighborhood', how='left')

zhvi_transit = pd.merge(zhvi_transit, const_trans, on='neighborhood', how='left')

zhvi_transit


,neighborhood,yymm,ZHVI,trans_score,SizeRank,RegionID,geometry,no_change,no_transit
0,MIRA MESA,1206,3.320540e+05,116.0,90,116625,MULTIPOLYGON (((6298262.993573308 1924450.0090...,0,0
1,RANCHO PENASQUITOS,1206,4.917199e+05,46.0,187,117557,MULTIPOLYGON (((6306168.141249999 1942019.0699...,0,0
2,CARMEL VALLEY,1206,6.741591e+05,0.0,204,273140,MULTIPOLYGON (((6279468.997368723 1927447.9994...,1,1
3,UNIVERSITY CITY,1206,3.971096e+05,109.0,207,118673,POLYGON ((6262511.028999999 1907533.1267500073...,0,0
4,RANCHO BERNARDO,1206,3.638746e+05,51.0,253,275127,MULTIPOLYGON (((6293013.998165801 1960702.6894...,0,0
...,...,...,...,...,...,...,...,...,...
17637,ISLENAIR,2503,7.449176e+05,6.0,13905,403241,"POLYGON ((6303394 1849792.9997500032, 6303394 ...",0,0
17638,HORTON PLAZA,2503,1.720785e+06,88.0,14394,268215,"POLYGON ((6281429 1840653, 6281427 1840265, 62...",0,0
17639,OLD TOWN,2503,1.082324e+06,95.0,15337,274893,POLYGON ((6272490.410999998 1857245.3289999962...,0,0
17640,BURLINGAME,2503,1.462991e+06,4.0,16412,403244,"POLYGON ((6292488.5390000045 1846444, 6292486 ...",0,0
